In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [1]:
import numpy as np  # linear algebra
import pandas as pd  #
from datetime import datetime
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import os

In [2]:
test=pd.read_csv('./data/test.csv')
train=pd.read_csv('./data/train.csv')
sample=pd.read_csv('./data/sample_submission.csv')
print(train.shape)
print(test.shape)
print(sample.shape)

(1460, 81)
(1459, 80)
(1459, 2)


In [3]:
##################删除训练集和测试集中的标签列-【开始】#################
#将train矩阵中的'Id'列删除（原地删除，故将inplace设为true），因为原始数据中的数据索引和预测模型的构建没有关系。
#test矩阵类似。
train.drop(['Id'], axis=1, inplace=True)
test.drop(['Id'], axis=1, inplace=True)
##################删除训练集和测试集中的标签列-【结束】#################

###############删除训练集中的极端值和进行数据index更新-【开始】#########
#使用条件筛选操作，通过覆值的方式剔除原始数据train矩阵中的极端值（极端值也被称为outliers），帮助预防房价预测模型出现过拟合。剔除操作也可以视为前剪枝。
train = train[train.GrLivArea < 4500]
#由于删去了部分行，故此时train矩阵中的index列并不连续。使用reset_index命令，在固定非index数据的顺序的前提下（inplace=True），重新对index编号（drop=True）。
train.reset_index(drop=True, inplace=True)
###############删除训练集中的极端值和进行数据index更新-【结束】#########

##########对预测目标数值进行对数变换和特征矩阵对象的创建-【开始】#######
# log1p就是log(1+x)，用来对房价数据进行数据预处理，它的好处是转化后的数据更加服从高斯分布，有利于后续的分类结果。
# 需要注意，最后需要将预测出的平滑数据还原，而还原过程就是log1p的逆运算expm1。
train["SalePrice"] = np.log1p(train["SalePrice"])
#单独取出训练数据中的房价列信息，存入y对象。
#y = train.SalePrice.reset_index(drop=True) #.reset_index(drop=True)方法：在原有的索引列重置索引，不再另外添加新列。有必要使用reset_index吗？有必要的，不这样做y将有两套index，作为df的y将有两列。
y = train['SalePrice'].reset_index(drop=True) #对上式的改写
#沿着水平的方向寻找列名为'SalePrice'的列（们），把它们对应的列统统删掉。得到了单纯的特征矩阵，存入train_features对象中。
train_features = train.drop(['SalePrice'], axis=1)
#test本来就没有房价列，所以它本来就是单纯的特征矩阵。
test_features = test
##########对预测目标数值进行对数变换和特征矩阵对象的创建-【结束】#######

##合并训练数据特征矩阵与测试数据特征矩阵，以便统一进行特征处理-【开始】##
#将训练数据中的特征矩阵和测试数据中的特征矩阵合并（.concat[矩阵1,矩阵2]），并对合并后的矩阵index重新编号（.reset_index(drop=True)）。
features = pd.concat([train_features, test_features]).reset_index(drop=True)
#检查合并后的矩阵的维数，核查合并结果。
print("剔除训练数据中的极端值后，将其特征矩阵和测试数据中的特征矩阵合并，维度为:",features.shape)
features.info()
##合并训练数据特征矩阵与测试数据特征矩阵，以便统一进行特征处理-【结束】##
#######################################################数据导入和特征提取-【结束】################################################################################


剔除训练数据中的极端值后，将其特征矩阵和测试数据中的特征矩阵合并，维度为: (2917, 79)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2917 entries, 0 to 2916
Data columns (total 79 columns):
MSSubClass       2917 non-null int64
MSZoning         2913 non-null object
LotFrontage      2431 non-null float64
LotArea          2917 non-null int64
Street           2917 non-null object
Alley            198 non-null object
LotShape         2917 non-null object
LandContour      2917 non-null object
Utilities        2915 non-null object
LotConfig        2917 non-null object
LandSlope        2917 non-null object
Neighborhood     2917 non-null object
Condition1       2917 non-null object
Condition2       2917 non-null object
BldgType         2917 non-null object
HouseStyle       2917 non-null object
OverallQual      2917 non-null int64
OverallCond      2917 non-null int64
YearBuilt        2917 non-null int64
YearRemodAdd     2917 non-null int64
RoofStyle        2917 non-null object
RoofMatl         2917 non-null object
Exterior1st   

In [4]:
###########把特征打印到文件，方便查看################
features.head()
features.to_csv("features.csv", index=False)
###########把特征打印到文件，方便查看 结束################

In [5]:
#############开始特征工程################# 
####首先进行缺失值的处理########
#
features['Functional'] = features['Functional'].fillna('Typ') #空值填充为str型数据'Typ'
features['Electrical'] = features['Electrical'].fillna("SBrkr") #空值填充为str型数据"SBrkr"
features['KitchenQual'] = features['KitchenQual'].fillna("TA") #空值填充为str型数据"TA"
features["PoolQC"] = features["PoolQC"].fillna("None") #空值填充为str型数据"None"

#对于列名为'Exterior1st'、'Exterior2nd'、'SaleType' "MSZoning"的特征列，使用列中的众数填充空值。
#	1.先查找数据列中的众数：使用df.mode()[]方法
#	  解释：df.mode(0或1,0表示对列查找，1表示对行查找)[需要查找众数的df列的index（就是df中的第几列）]，将返回数据列中的众数
#	2.使用.fillna()方法进行填充
features['Exterior1st'] = features['Exterior1st'].fillna(features['Exterior1st'].mode()[0]) 
features['Exterior2nd'] = features['Exterior2nd'].fillna(features['Exterior2nd'].mode()[0])
features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])
features['MSZoning']=features['MSZoning'].fillna(features['MSZoning'].mode()[0])



#对于列名为'GarageYrBlt', 'GarageArea', 'GarageCars'的特征列，使用0填充空值。表示没有GARAGE。
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    features[col] = features[col].fillna(0)

#对于列名为'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'的特征列，使用字符串'None'填充空值。表示没有GARAGE。
for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']:
    features[col] = features[col].fillna('None')
    
#对于列名为'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'的特征列，使用字符串'None'填充空值。表示没有BASEMENT.
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    features[col] = features[col].fillna('None')
    
#使用传入矩阵（'LotFrontage'列）的中位数对传入矩阵中的空值进行填充。
#先以'Neighborhood'为标签，以'LotFrontage'为被汇总序列。然后使用被汇总序列中的中位数，对原始矩阵'LotFrontage'列中的空值进行填充。
#transform的特性是同维操作，最后输出结果的顺序和原始数据在序号上完全匹配。
features['LotFrontage'] = features.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

#对于整型和浮点型数据列，使用0填充其中的空值。
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics = []
for i in features.columns:
    if features[i].dtype in numeric_dtypes:
        numerics.append(i)
features.update(features[numerics].fillna(0))

######################数字型数据列偏度校正-【开始】#######################
#使用skew()方法，计算所有整型和浮点型数据列中，数据分布的偏度（skewness）。
#偏度是统计数据分布偏斜方向和程度的度量，是统计数据分布非对称程度的数字特征。亦称偏态、偏态系数。 
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics2 = []
for i in features.columns:
    if features[i].dtype in numeric_dtypes:
        numerics2.append(i)
skew_features = features[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)

#以0.5作为基准，统计偏度超过此数值的高偏度分布数据列，获取这些数据列的index。
high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

#对高偏度数据进行处理，将其转化为正态分布。
#Box和Cox提出的变换可以使线性回归模型满足线性性、独立性、方差齐次以及正态性的同时，又不丢失信息。
for i in skew_index:
    features[i] = boxcox1p(features[i], boxcox_normmax(features[i] + 1))#这是boxcox1p的使用方法，参数的具体意义暂时不解释
######################数字型数据列偏度校正-【结束】#######################


#判断出features矩阵中列为对象的列，将列名存入objects叔祖。对于features矩阵中的各个列对象，将其列中的空值填充为'None'
objects = []
for i in features.columns:
    if features[i].dtype == object:
        objects.append(i)
features.update(features[objects].fillna('None')) 

#简化特征。对于某些分布单调（比如100个数据中有99个的数值是0.9，另1个是0.1）的数字型数据列，进行01取值处理。
features['haspool'] = features['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
features['has2ndfloor'] = features['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
features['hasgarage'] = features['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
features['hasbsmt'] = features['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
features['hasfireplace'] = features['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)



/Applications/anaconda3/envs/dash/lib/python3.7/site-packages/scipy/stats/stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/Applications/anaconda3/envs/dash/lib/python3.7/site-packages/scipy/stats/stats.py:3429: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficent may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


In [6]:
features.info()
#features.to_csv("features3.csv", index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2917 entries, 0 to 2916
Data columns (total 84 columns):
MSSubClass       2917 non-null float64
MSZoning         2917 non-null object
LotFrontage      2917 non-null float64
LotArea          2917 non-null float64
Street           2917 non-null object
Alley            2917 non-null object
LotShape         2917 non-null object
LandContour      2917 non-null object
Utilities        2917 non-null object
LotConfig        2917 non-null object
LandSlope        2917 non-null object
Neighborhood     2917 non-null object
Condition1       2917 non-null object
Condition2       2917 non-null object
BldgType         2917 non-null object
HouseStyle       2917 non-null object
OverallQual      2917 non-null int64
OverallCond      2917 non-null float64
YearBuilt        2917 non-null int64
YearRemodAdd     2917 non-null int64
RoofStyle        2917 non-null object
RoofMatl         2917 non-null object
Exterior1st      2917 non-null object
Exterior2nd      2

In [7]:
######################特征删除和融合创建新特征-【开始】###################
#删除一些特征。df.drop（‘列名’, axis=1）代表将‘列名’对应的列标签（们）沿着水平的方向依次删掉。
#features = features.drop(['Utilities', 'Street', 'PoolQC',], axis=1)

In [8]:
#########对下面几个特征进行分列#######
objects = []
for i in features.columns:
    if features[i].dtype == object:
        temp = pd.get_dummies(features[i], prefix=features[[i]].columns[0])
        features = pd.concat([features, temp], axis=1)

for i in features.columns:
    if features[i].dtype == object:
        features = features.drop([i], axis=1)
MSSubClass = pd.get_dummies(features['MSSubClass'], prefix=features[['MSSubClass']].columns[0])
features = pd.concat([features, MSSubClass], axis=1)

YrSold = pd.get_dummies(features['YrSold'], prefix=features[['YrSold']].columns[0])
features = pd.concat([features, YrSold], axis=1)

MoSold = pd.get_dummies(features['MoSold'], prefix=features[['MoSold']].columns[0])
features = pd.concat([features, MoSold], axis=1)
features.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12
0,6.499671,18.144572,13.833054,7,3.991517,2003,2003,19.433172,144.117870,0.0,...,0,0,0,0,0,0,0,0,0,0
1,4.249693,20.673624,14.117918,6,6.000033,1976,1976,0.000000,181.719196,0.0,...,0,0,1,0,0,0,0,0,0,0
2,6.499671,18.668046,14.476513,7,3.991517,2001,2002,17.768838,110.441039,0.0,...,0,0,0,0,0,0,1,0,0,0
3,6.862539,17.249650,14.106197,7,3.991517,1915,1970,0.000000,61.795317,0.0,...,0,0,0,0,0,0,0,0,0,0
4,6.499671,21.314282,15.022008,8,3.991517,2000,2000,25.404161,136.624608,0.0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
#融合多个特征，生成新特征。
features['YrBltAndRemod']=features['YearBuilt']+features['YearRemodAdd']
features['TotalSF']=features['TotalBsmtSF'] + features['1stFlrSF'] + features['2ndFlrSF']

features['Total_sqr_footage'] = (features['BsmtFinSF1'] + features['BsmtFinSF2'] +
                                 features['1stFlrSF'] + features['2ndFlrSF'])

features['Total_Bathrooms'] = (features['FullBath'] + (0.5 * features['HalfBath']) +
                               features['BsmtFullBath'] + (0.5 * features['BsmtHalfBath']))

features['Total_porch_sf'] = (features['OpenPorchSF'] + features['3SsnPorch'] +
                              features['EnclosedPorch'] + features['ScreenPorch'] +
                              features['WoodDeckSF'])

features['yearslived'] = features['YrSold'] - features['YearBuilt']
features['yearsremod'] = features['YrSold'] - features['YearRemodAdd']


In [10]:
#还原到测试集和训练集
train_data_X = features.iloc[:len(y), :]	#y是列向量，存储了训练数据中的房价列信息。截取后得到的X阵的维度是len(y)*(final_features的列数)。
test_data_X = features.iloc[len(y):, :]#使用len命令，求矩阵X的长度，得到的是矩阵对象的长度，即有矩阵中有多少列，而不是每列上有多少行。


#在新生特征空间中，剔除X阵和y阵中有着极端值的各行数据（因为X和y阵在水平方向上是一致的，所以要一起删除同样的行）。outliers数值中给出了极端值的列序号。
#df.drop(df.index[序号])将删除指定序号的各行。再使用=对df覆值。
outliers = [30, 88, 462, 631, 1322]
train_data_X = train_data_X.drop(train_data_X.index[outliers])#因为X阵是经过对特征矩阵进行类似“坐标投影”操作后得到的，列向量y中的行号对应着X阵中的列号。
y = y.drop(y.index[outliers])

train_data_y = y

In [11]:
features.info()
train_data_X.info()
test_data_X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2917 entries, 0 to 2916
Columns: 347 entries, MSSubClass to yearsremod
dtypes: float64(33), int64(15), uint8(299)
memory usage: 1.9 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1453 entries, 0 to 1457
Columns: 347 entries, MSSubClass to yearsremod
dtypes: float64(33), int64(15), uint8(299)
memory usage: 980.5 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 1458 to 2916
Columns: 347 entries, MSSubClass to yearsremod
dtypes: float64(33), int64(15), uint8(299)
memory usage: 973.3 KB


# 下面用几个模型来筛选比较重要的特征

In [12]:
#############个体机器学习模型的创建（即模型声明和参数设置）-【开始】############
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

#定义ridge岭回归模型（使用二范数作为正则化项。不论是使用一范数还是二范数，正则化项的引入均是为了降低过拟合风险。）
#注：正则化项如果使用二范数，那么对于任何需要寻优的参数值，在寻优终止时，它都无法将某些参数值变为严格的0，尽管某些参数估计值变得非常小以至于可以忽略。即使用二范数会保留变量的所有信息，不会进行类似PCA的变量凸显。
#注：正则化项如果使用一范数，它比L2范数更易于获得“稀疏(sparse)”解，即它的求解结果会有更多的零分量。
# ridge = make_pipeline(RobustScaler(), RidgeCV(alphas=alphas_alt, cv=kfolds))

#定义LASSO收缩模型（使用L1范数作为正则化项）（由于对目标函数的求解结果中将得到很多的零分量，它也被称为收缩模型。）
#注：正则化项如果使用二范数，那么对于任何需要寻优的参数值，在寻优终止时，它都无法将某些参数值变为严格的0，尽管某些参数估计值变得非常小以至于可以忽略。即使用二范数会保留变量的所有信息，不会进行类似PCA的变量凸显。
#注：正则化项如果使用一范数，它比L2范数更易于获得“稀疏(sparse)”解，即它的求解结果会有更多的零分量。										
# lasso = make_pipeline(RobustScaler(), LassoCV(max_iter=1e7, alphas=alphas2, random_state=42, cv=kfolds))

#定义elastic net弹性网络模型（弹性网络实际上是结合了岭回归和lasso的特点，同时使用了L1和L2作为正则化项。）									
elasticnet = make_pipeline(RobustScaler(), ElasticNetCV(max_iter=1e7, alphas=e_alphas, cv=5, l1_ratio=e_l1ratio))

#定义random forest model									
rf = RandomForestRegressor(n_estimators=3000, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, criterion='mse', random_state =42)                             

#定义GB梯度提升模型（展开到一阶导数）									
gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, loss='huber', random_state =42)                             

#定义lightgbm模型									
lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=5000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       #min_data_in_leaf=2,
                                       #min_sum_hessian_in_leaf=11
                                       )

#定义xgboost模型（展开到二阶导数）                                      
xgboost = XGBRegressor(learning_rate=0.01, n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:squarederror', nthread=4,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)
#############个体机器学习模型的创建（即模型声明和参数设置）-【结束】############

In [13]:
def get_top_n_features(train_data_X, train_data_y, top_n_features,models = []):
        # 随机森林
    #定义xgboost模型（展开到二阶导数）  
    features_top_n = []
    for model in models:
        
#        scorings = -cross_val_score(model, train_data_X, train_data_y, cv=5, 
#                    scoring='neg_mean_squared_error',n_jobs=4)
#        rmse = np.sqrt(scorings).mean()
#        print(rmse)
        model.fit(train_data_X, train_data_y)
        #将feature按Importance排序
        feature_imp_sorted_each = pd.DataFrame({'feature': list(train_data_X), 'importance': model.feature_importances_}).sort_values('importance', ascending=False)
        features_top_n_each = feature_imp_sorted_each.head(top_n_features)['feature']
        print('Sample 25 Features from Classifier')
        print(str(features_top_n_each[:25]))
        if(len(features_top_n) == 0):
            features_top_n = features_top_n_each
        features_top_n = pd.concat([features_top_n,features_top_n_each], ignore_index=True).drop_duplicates()
    



    

    return features_top_n

In [14]:
feature_to_pick = 250
models = [rf,gbr,lightgbm,xgboost]
feature_top_n = get_top_n_features(train_data_X,train_data_y,feature_to_pick,models = models)
picked_train_data_X = train_data_X[feature_top_n]
picked_test_data_X = test_data_X[feature_top_n]

Sample 25 Features from Classifier
3           OverallQual
15            GrLivArea
345          yearslived
5             YearBuilt
343     Total_Bathrooms
340       YrBltAndRemod
25           GarageCars
176        ExterQual_TA
11          TotalBsmtSF
26           GarageArea
12             1stFlrSF
24          GarageYrBlt
18             FullBath
341             TotalSF
40         hasfireplace
346          yearsremod
249    FireplaceQu_None
238      KitchenQual_TA
23           Fireplaces
184    Foundation_PConc
6          YearRemodAdd
175        ExterQual_Gd
188         BsmtQual_Ex
22         TotRmsAbvGrd
2               LotArea
Name: feature, dtype: object
Sample 25 Features from Classifier
343      Total_Bathrooms
15             GrLivArea
340        YrBltAndRemod
3            OverallQual
11           TotalBsmtSF
238       KitchenQual_TA
176         ExterQual_TA
26            GarageArea
18              FullBath
25            GarageCars
6           YearRemodAdd
12              1stFlrSF
2

/Applications/anaconda3/envs/dash/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Sample 25 Features from Classifier
25                       GarageCars
23                       Fireplaces
40                     hasfireplace
188                     BsmtQual_Ex
249                FireplaceQu_None
3                       OverallQual
268                   GarageQual_TA
343                 Total_Bathrooms
15                        GrLivArea
340                   YrBltAndRemod
11                      TotalBsmtSF
253               GarageType_Attchd
173                    ExterQual_Ex
174                    ExterQual_Fa
228                    CentralAir_N
45                      MSZoning_RM
238                  KitchenQual_TA
229                    CentralAir_Y
308    MSSubClass_5.014201084298206
140             Exterior1st_BrkComm
235                  KitchenQual_Ex
76             Neighborhood_Crawfor
283                      Fence_GdWo
244                  Functional_Sev
220                    Heating_Grav
Name: feature, dtype: object


# 下面是STACKING 学习

In [15]:
#定义均方根对数误差（Root Mean Squared Logarithmic Error ，RMSLE）
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))


#创建模型评分函数，根据不同模型的表现打分
#cv表示Cross-validation,交叉验证的意思。
def cv_rmse(model, X=picked_train_data_X,y = train_data_y ):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error",cv = 5,n_jobs=4))
    return (rmse)
from sklearn.ensemble import VotingRegressor
#################投票集成 ##################
#ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3)])

###########################集成多个个体学习器-【开始】##########################
stack_gen = StackingCVRegressor(regressors=(elasticnet, gbr, xgboost, lightgbm),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)#regressors=(...)中并没有纳入前面的svr模型
###########################集成多个个体学习器-【结束】##########################                             

############################进行交叉验证打分-【开始】###########################
#进行交叉验证，并对不同模型的表现打分
#（由于是交叉验证，将使用不同的数据集对同一模型进行评分，故每个模型对应一个得分序列。展示模型得分序列的平均分、标准差）
print('进行交叉验证，计算不同模型的得分TEST score on CV')

#打印elastic net弹性网络模型的得分
score = cv_rmse(elasticnet)
print("elastic net弹性网络模型的得分: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

#打印lightgbm轻梯度提升模型的得分
score = cv_rmse(lightgbm)
print("lightgbm轻梯度提升模型的得分: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

#打印gbr梯度提升回归模型的得分
score = cv_rmse(gbr)
print("gbr梯度提升回归模型的得分: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

#打印随机森林模型的得分
score = cv_rmse(rf)
print("rf随机森林模型的得分: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

#打印xgboost模型的得分
score = cv_rmse(xgboost)
print("xgboost模型的得分: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )
############################进行交叉验证打分-【结束】###########################

#########使用训练数据特征矩阵作为输入，训练数据对数处理后的预测房价作为输出，进行各个模型的训练-【开始】#########
#开始集合所有模型，使用stacking方法
print('进行模型参数训练 START Fit')

print(datetime.now(), '对stack_gen集成器模型进行参数训练')
stack_gen_model = stack_gen.fit(np.array(picked_train_data_X), np.array(train_data_y))

print(datetime.now(), '对elasticnet弹性网络模型进行参数训练')
elastic_model_full_data = elasticnet.fit(picked_train_data_X, train_data_y)

print(datetime.now(), '对GradientBoosting梯度提升模型进行参数训练')
gbr_model_full_data = gbr.fit(picked_train_data_X, train_data_y)

print(datetime.now(), '对xgboost二阶梯度提升模型进行参数训练')
xgb_model_full_data = xgboost.fit(picked_train_data_X, train_data_y)

print(datetime.now(), '对lightgbm轻梯度提升模型进行参数训练')
lgb_model_full_data = lightgbm.fit(picked_train_data_X, train_data_y)

print(datetime.now(), '对rf轻梯度提升模型进行参数训练')
rf_model_full_data = rf.fit(picked_train_data_X, train_data_y)
#########使用训练数据特征矩阵作为输入，训练数据对数处理后的预测房价作为输出，进行各个模型的训练-【结束】#########

############################进行交叉验证打分-【结束】###########################

进行交叉验证，计算不同模型的得分TEST score on CV
elastic net弹性网络模型的得分: 0.1011 (0.0058)
 2019-12-29 17:17:26.411762
lightgbm轻梯度提升模型的得分: 0.1085 (0.0048)
 2019-12-29 17:17:31.193905
gbr梯度提升回归模型的得分: 0.1103 (0.0088)
 2019-12-29 17:17:57.174081
rf随机森林模型的得分: 0.1841 (0.0119)
 2019-12-29 17:18:14.616052
xgboost模型的得分: 0.1060 (0.0043)
 2019-12-29 17:19:07.467209
进行模型参数训练 START Fit
2019-12-29 17:19:07.467487 对stack_gen集成器模型进行参数训练
2019-12-29 17:22:06.804010 对elasticnet弹性网络模型进行参数训练
2019-12-29 17:22:09.012141 对GradientBoosting梯度提升模型进行参数训练
2019-12-29 17:22:21.039077 对xgboost二阶梯度提升模型进行参数训练


/Applications/anaconda3/envs/dash/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


2019-12-29 17:22:38.963047 对lightgbm轻梯度提升模型进行参数训练
2019-12-29 17:22:40.348754 对rf轻梯度提升模型进行参数训练


In [19]:
#### 这 blend的 参数值是咋决定的？
########定义个体学习器的预测值融合函数，检测预测值融合策略的效果-【开始】#######
#综合多个模型产生的预测值，作为多模型组合学习器的预测值
def blend_models_predict(X):
    return ((0.1 * elastic_model_full_data.predict(X)) + \
#             (0.05 * lasso_model_full_data.predict(X)) + \
#             (0.1 * ridge_model_full_data.predict(X)) + \
#             (0.1 * svr_model_full_data.predict(X)) + \
            (0.1 * gbr_model_full_data.predict(X)) + \
            (0.15 * xgb_model_full_data.predict(X)) + \
            (0.1 * lgb_model_full_data.predict(X)) + \
            (0.55 * stack_gen_model.predict(np.array(X))))

#打印在上述模型配比下，多模型组合学习器的均方根对数误差（Root Mean Squared Logarithmic Error ，RMSLE）
#使用训练数据对创造的模型进行k折交叉验证，以训练创造出的模型的参数配置。交叉验证训练过程结束后，将得到模型的参数配置。使用得出的参数配置下，在全体训练数据上进行验证，验证模型对全体训练数据重构的误差。
print('融合后的训练模型对原数据重构时的均方根对数误差RMSLE score on train data:')
print(rmsle(train_data_y, blend_models_predict(picked_train_data_X)))
########定义个体学习器的预测值融合函数，检测预测值融合策略的效果-【结束】#######

########将测试集的特征矩阵作为输入，传入训练好的模型，得出的输出写入.csv文件的第2列-【开始】########
print('使用测试集特征进行房价预测 Predict submission', datetime.now(),)
submission = pd.read_csv("./submission/submission3_baseline.csv")
#函数注释：.iloc[:,1]是基于索引位来选取数据集，[索引1:索引2]，左闭右开。
submission.iloc[:,1] = np.floor(np.expm1(blend_models_predict(picked_test_data_X)))
########将测试集的特征矩阵作为输入，传入训练好的模型，得出的输出写入.csv文件的第2列-【结束】########
#以csv文件的形式输出预测值
submission.to_csv("./submission/House_price_submission_ML1.csv", index=False)
print('融合结果.csv文件输出成功 Save submission', datetime.now())

融合后的训练模型对原数据重构时的均方根对数误差RMSLE score on train data:
0.04121973119132323
使用测试集特征进行房价预测 Predict submission 2019-12-29 17:28:03.417952
融合结果.csv文件输出成功 Save submission 2019-12-29 17:28:04.456863
